# CS 4100 Homework 02 

#### Due Thursday  2/2 at midnight (1 minute after 11:59 pm) in Gradescope (with a grace period of 6 hours)
#### You may submit the homework up to 24 hours late (with the same grace period) for a penalty of 10%. 

You must submit the homework in Gradescope as a zip file containing <b>two files:</b>
- The <code>.ipynb</code> file (be sure to <code>Kernel -> Restart and Run All</code> before you submit); and
- A <code>.pdf</code> file of the notebook. 

For best results obtaining a clean PDF file on the Mac, select <code>File -> Print Review</code> from the Jupyter window, then choose <code>File-> Print</code> in your browser and then <code>Save as PDF</code>.  Something  similar should be possible on a Windows machine. 


 
All homeworks will be scored with a maximum of 100 points; if point values are not given for individual
problems, then all problems will be counted equally. 

Note: I have uploaded my solution for HW 01 on the class web page if you wish to use it; some of the code in HW 01 will be needed in this homework. 

Here are some useful utility functions for examining the results of your code. 

In [1]:
# This will pretty-print literals, clauses, and clause lists.
# It will print out propositional letters as A, B, C, ...
    
def literal2String(L):
    return ('-' if (L<0) else '') + chr(ord('A')+abs(L)-1)

def clause2String(C):
    if(C=={}):
        return '{}'
    else:
        return '{ '+(', '.join([literal2String(L) for L in C]))+' }'

def clauseList2String(A):
    return '[ '+(', '.join([clause2String(C) for C in A]))+' ]'

def pprint(X):
    if(type(X)==int):              # X is literal
        print(literal2String(X))
    elif(type(X)==set):            # X is a clause
        print(clause2String(X))
    elif(type(X)==list):           # X is a clause list
        print(clauseList2String(X))
    else:
        print('Error in pprint!')

        #test

pprint(-2)
pprint({3,-2})
pprint([ {1},{-1,2}, {-2,-1,3} ])
pprint([ {1}, {}, {-2} ])


-B
{ C, -B }
[ { A }, { B, -A }, { C, -B, -A } ]
[ { A }, {}, { -B } ]


## Problem One: Resolution Theorem Proving in Propositional Logic (30 pts)

In this problem we will explore the resolution calculus, using breadth-first search to find the empty clause (if it exists) in the resolvents generated from the input set. 

### Part A: Resolution Rule

Complete the following template according to the definition of resolution given in your text and in lecture. 

In [2]:
# Return all possible resolvents of clauses C1 and C2 (sets)

def resolve(C1,C2):
    resolvents = []
    for literal1 in C1:
        for literal2 in C2:
            if literal1 == -literal2:
                resolvent = (C1 - {literal1}) | (C2 - {literal2})
                if resolvent not in resolvents:
                    resolvents.append(resolvent)
    return resolvents
                
 
# Tests -- Note that your sets may occur in a different order than shown here

C1 = {1}
C2 = {-1}
print(clause2String(C1),clause2String(C2))                      
print(clauseList2String(resolve(C1,C2)),'\n')   # should be [ {} ]

C1 = {1,2}
C2 = {-1,-3}
print(clause2String(C1),clause2String(C2))                      
print(clauseList2String(resolve(C1,C2)),'\n')   # should be [ { B, -C } ]

C1 = {}
C2 = {1,-2}
print(clause2String(C1),clause2String(C2))                      
print(clauseList2String(resolve(C1,C2)),'\n')   # should be [  ]

C1 = {1,-3}
C2 = {1,-2}
print(clause2String(C1),clause2String(C2))                      
print(clauseList2String(resolve(C1,C2)),'\n')   # should be [  ]

C1 = {1,2,3}
C2 = {-1,-2,-3}
print(clause2String(C1),clause2String(C2))                      
print(clauseList2String(resolve(C1,C2)),'\n')   # should be [ { B, C, -C, -B }, { A, C, -C, -A }, { A, B, -A, -B } ]

{ A } { -A }
[ {  } ] 

{ A, B } { -C, -A }
[ { B, -C } ] 

{} { A, -B }
[  ] 

{ A, -C } { A, -B }
[  ] 

{ A, B, C } { -C, -A, -B }
[ { B, C, -C, -B }, { A, C, -C, -A }, { A, B, -A, -B } ] 



### Part B: Resolution Rule Continued

Complete the following template according to the definition of resolution given in your text and in lecture. 

In [3]:
# A is a list of clauses, C is a clause
# Return list of all possible resolvents of C with a clause in A

def resolveAll(A,C):   
    resolvents = []
    for clause in A:
        new_resolvents = resolve(clause, C)
        resolvents.extend(new_resolvents)
    return resolvents

# tests

KB = [ {-2,-1,3} ]
NQ = {-3}

pprint(KB)
pprint(NQ)
pprint(resolveAll(KB,NQ))     # should be [ { -B, -A } ]
print()

KB = [ {1}, {}, {-2,-1,-3} ]
NQ = {-3}

pprint(KB)
pprint(NQ)
pprint(resolveAll(KB,NQ))     # should be [  ]
print()

KB = [ ]
NQ = {-3}

pprint(KB)
pprint(NQ)
pprint(resolveAll(KB,NQ))     # should be [  ]
print()

KB = [ {1}, {-1,2}, {-2,-1,3} ]
NQ = {-3,1}

pprint(KB)
pprint(NQ)
pprint(resolveAll(KB,NQ))     # should be [ { B, -C }, { A, -B, -A }, { C, -C, -B } ]

[ { C, -B, -A } ]
{ -C }
[ { -B, -A } ]

[ { A }, {}, { -C, -B, -A } ]
{ -C }
[  ]

[  ]
{ -C }
[  ]

[ { A }, { B, -A }, { C, -B, -A } ]
{ A, -C }
[ { B, -C }, { A, -B, -A }, { C, -C, -B } ]


### Part C:  Breadth-first Prover

Write a function `prove1` which takes a set `CL`
and searches for the empty clause by inserting all members of CL into a queue, and generating
resolvents from the clause popped off the front of the queue with any member of the rest of queue, 
and adding them to the back of the queue. It should behave as shown below:

- If the empty clause is found, report "Unsatisfiable";
- If the queue empties out, report "Satisfiable"; 
- If the prover can not find the empty clause after 'limit' steps, report that fact. 

In each case, you should also report the number of steps (= times you popped the front off the queue);

If the parameter `trace` is true, you should print out the queue at each step. 

In [4]:
# perform breath-first search from a set of clauses CL, by
# inserting all of CL into a queue
# and then searching using BFS by generating resolvents of head of queue with any
# clause in the queue

def prove1(CL, limit=30, trace=True):
    queue = CL
    visited = set()
    steps = 0
    while queue and steps < limit:
        steps += 1
        current = queue.pop(0)
        if trace:
            print(f"Step {steps}: {current}")
        for resolvent in resolveAll(queue, current):
            if len(resolvent) == 0:
                print(f"Unsatisfiable. Number of steps: {steps}")
                return
            elif frozenset(resolvent) not in visited:
                queue.append(resolvent)
                visited.update(frozenset(resolvent))
    if steps == limit:
        print(f"Not proven after {limit} steps.")
    else:
        print(f"Satisfiable. Number of steps: {steps}")
  

In [5]:
# tests: first 3 are unsatisfiable, test 4 is satisfiable

print('\ntest a','\n------')
CL1a = [ {2}, {-2} ]     

print('CL1a: ',end=''); pprint(CL1a); print()
prove1(CL1a); print()

print('test b','\n------')
CL1b = [ {1}, {-1,2}, {-2,3}, {-3,4}, {-4} ]

print('CL1b: ',end=''); pprint(CL1b); print()
prove1(CL1b); print()

print('test c','\n------')
CL1c = [ {1}, {-1,2,3}, {-3,4}, {-3,5}, {-5}, {-2} ]

print('CL1c: ',end=''); pprint(CL1c); print()
prove1(CL1c); print()

print('test d','\n------')
CL1d = [ {1}, {-1,2,3}, {-3,4}, {-3,5}, {-5} ]

print('CL1d: ',end=''); pprint(CL1d); print()
prove1(CL1d); print()



test a 
------
CL1a: [ { B }, { -B } ]

Step 1: {2}
Unsatisfiable. Number of steps: 1

test b 
------
CL1b: [ { A }, { B, -A }, { C, -B }, { D, -C }, { -D } ]

Step 1: {1}
Step 2: {2, -1}
Step 3: {3, -2}
Step 4: {4, -3}
Step 5: {-4}
Unsatisfiable. Number of steps: 5

test c 
------
CL1c: [ { A }, { B, C, -A }, { D, -C }, { E, -C }, { -E }, { -B } ]

Step 1: {1}
Step 2: {2, 3, -1}
Step 3: {4, -3}
Step 4: {5, -3}
Step 5: {-5}
Step 6: {-2}
Unsatisfiable. Number of steps: 6

test d 
------
CL1d: [ { A }, { B, C, -A }, { D, -C }, { E, -C }, { -E } ]

Step 1: {1}
Step 2: {2, 3, -1}
Step 3: {4, -3}
Step 4: {5, -3}
Step 5: {-5}
Step 6: {2, 3}
Step 7: {2, 4, -1}
Step 8: {2, 5, -1}
Step 9: {2, 4}
Step 10: {-3}
Step 11: {2, 5}
Step 12: {2, -1}
Step 13: {2}
Step 14: {2}
Satisfiable. Number of steps: 14



## Problem Two (10 pts)

In this problem, we will experiment with our prover to understand its behavior. 

### Part A

Generate a list of clauses `CL2a` such that

-  CL2a is satisfiable, but 
-  It will make the prover run forever (in this case, not terminating with "Satisfiable!" within 30 steps)

You must 

- Pretty-print CL2a
- Prove that it is satisfiable (using your code from HW 01), 
- Show that it fails to terminate after 30 steps, and
- Explain why your example behaves in this way. 

Note: Your example should be such that no matter how large `limit` is, it will not terminate; however,
for purposes of grading, you should leave `limit=30` but `trace=False`.  

Hint: See if you can get the prover to loop uselessly on the same clauses. It may help to
think about the order in which clauses will be processed. 

In [6]:
from itertools import product

def evalLiteral(I,L):
    return I[abs(L)-1] if L > 0 else not I[abs(L)-1]

# C represents a clause as an array
def evalClause(I,C):
    satisfied = False
    for literal in C:
        satisfied = evalLiteral(I, literal) or satisfied
        if satisfied:
            break
    return satisfied

# A is the CNF as a 2D array
def evalCNF(A):
    isValid = True;
    isSatisfiable = False;
    
    literalSet = set()
    for clause in A:
        for literal in clause:
            literalSet.add(abs(literal))
    literalTable = list(product([True, False], repeat=len(literalSet)))

    for I in literalTable:
        clauseEval = True
        for clause in A:
            clauseEval = evalClause(I, clause) and clauseEval

        isValid = clauseEval and isValid
        isSatisfiable = clauseEval or isSatisfiable
    
    if isValid:
        print("Valid")
    elif isSatisfiable:
        print("Satisfiable")
    else:
        print("Unsatisfiable")

In [7]:
CL2a = [{1, -2}, {2, -1}, {-1, -2}]
print('CL2a: ',end=''); pprint(CL2a); print()
evalCNF(CL2a)
prove1(CL2a, 30, False); print()

# The way these clauses are set up cause the prover to loop infinitely through different variations of the clauses given.
# After step 31 the program will loop infinitely on {-1}, never to complete without the limit.
# The first 31 steps loop through resolvents of {-2, -1}, {-1}, {-2}, {2, -1}, {2, -2}, {1, -1}, {1, -2}

CL2a: [ { A, -B }, { B, -A }, { -A, -B } ]

Satisfiable
Not proven after 30 steps.



### Part B  

In this problem we will explore what happens when the clause set causes an exponential explosion 
in the search space. We will use a classic example of a clause set which requires exponential time (assuming
$P\ne NP$). 

First create a clause list `CL2b` from only 2 symbols which is unsatisfiable but causes
exponential behavior. The easiest way to do this is to create a clause set with 4 clauses, each of
which makes one row of the truth table for two symbols (say A and B) false. For example,
for a row that has **(True,False)**, you would create $\{ \neg A, B \} = \neg A\vee B.$ If you
make sure that no row of the truth table is a model, then it must be unsatisfiable, but
each clause is necessary to prove this fact (if you leave a clause out, then that row
of the table would be a model). 

It should take 7 steps to determine unsatisfiability. 

As in Part A, 

- Pretty-print CL2b
- Verify that it is unsatisfiable using code from HW 01
- Show that it takes 7 steps to find the empty clause. 


In [8]:
CL2b = [{-1, -2}, {-1, 2}, {1, -2}, {1, 2}]
print('CL2b: ',end=''); pprint(CL2b); print()
evalCNF(CL2b)
prove1(CL2b, 30, False); print()

CL2b: [ { -A, -B }, { B, -A }, { A, -B }, { A, B } ]

Unsatisfiable
Unsatisfiable. Number of steps: 5



### Part C

Now, create a clause list `CL2d`from 3 symbols, with 8 clauses, along the same lines as the previous example (each
clause makes one line of the truth table false).  

 

To do:

- Pretty-print CL2d
- Prove that it is unsatisfiable as above
- Run this with a limit of 3, 4, and 5 to get a sense for what this will do. Then turn off tracing and try it for a few more (very soon it will simply get lost in an exponential explosion).
- Answer this question: How large could you make limit and still get "Empty clause not found after ..." to print out within say a minute?

In [9]:
CL2d = [{-1, -2, -3}, {-1, -2, 3}, {-1, 2, -3}, {-1, 2, 3}, {1, -2, -3}, {1, -2, 3}, {1, 2, -3}, {1, 2, 3}]
print('CL2d: ',end=''); pprint(CL2d); print()
evalCNF(CL2d); print()
# prove1(CL2d, 3, True); print()
# prove1(CL2d, 4, True); print()
# prove1(CL2d, 5, True); print()

prove1(CL2d, 15, False); print()
# prove1(CL2d, 20, False); print()

# Anything past 15 seems to crash my computer when run in jupyter notebooks.
# When running the code just straight up in a python file, a limit of 20 finishes around the one minute mark

CL2d: [ { -C, -A, -B }, { C, -A, -B }, { B, -C, -A }, { B, C, -A }, { A, -C, -B }, { A, C, -B }, { A, B, -C }, { A, B, C } ]

Unsatisfiable

Not proven after 15 steps.



## Refinements of Resolution

There are various ways to make resolution more efficient, including

- Set of Support Strategy: partition your clause set into subsets KB and SOS, such that KB is satisfiable, but KB + SOS is unsatisfiable; 
- Removing useless clauses such as tautologies and duplicates
- Shortest-Clauses-First: use a priority queue instead of a queue, and keep it ordered by size, with
smaller clauses to the front.

The motivation for all of these is to prevent doing useless work, and--in the case of the shortest-first--to try to focus on creating short clauses as soon as possible. 


## Problem 3 (Set of Support) (10 pts)


### Part A 

Rewrite your prover to create a function `prove3` which uses the set of support strategy, 
where the set of unsatisfiable clauses is separated into KB and SOS, in such a way that KB is
known to be satisfiable. 

- Your prover will take KB and SOS as separate arguments;
- Create your initial queue from SOS alone;
- In each step, remove the head of the queue, and form resolvents between that clause
and any clause in KB + Queue. 

Rewrite the tests from Problem One, Part A, such that the *last* clause in each list is removed
and put in the SOS. The first one is done for you to show what is needed. 

Run your prover on these four examples to verify that all is working as expected. 

In [10]:
def prove3(KB, SOS, limit=30, trace=True):
    queue = SOS
    visited = set()
    steps = 0
    while queue and steps < limit:
        steps += 1
        current = queue.pop(0)
        if trace:
            print(f"Step {steps}: {current}")
        
        if not current:
            print(f"Unsatisfiable. Number of steps: {steps}")
            return
        
        for resolvent in resolveAll(KB + queue, current):
            if len(resolvent) == 0:
                print(f"Unsatisfiable. Number of steps: {steps}")
                return
            elif frozenset(resolvent) not in visited:
                queue.append(resolvent)
                visited.update(frozenset(resolvent))

    if steps == limit:
        print(f"Not proven after {limit} steps.")
    else:
        print(f"Satisfiable. Number of steps: {steps}")

In [11]:
print('\ntest a','\n------')
KB3a1 = [ {2} ]
SOS3a1 = [ {-2} ]

print('KB3a1: ',end=''); pprint(KB3a1); 
print('SOS3a1: ',end=''); pprint(SOS3a1); print()
prove3(KB3a1,SOS3a1); print()

print('test b','\n------')
KB3a2 = [ {1}, {-1,2}, {-2,3}, {-3,4} ]
SOS3a2 = [ {-4} ]

print('KB3a2: ',end=''); pprint(KB3a2);
print('SOS3a2: ',end=''); pprint(SOS3a2); print()
prove3(KB3a2,SOS3a2); print()

print('test c','\n------')
KB3a3 = [ {1}, {-1,2,3}, {-3,4}, {-3,5}, {-5} ]
SOS3a3 = [ {-2} ]

print('KB3a3: ',end=''); pprint(KB3a3);
print('SOS3a3: ',end=''); pprint(SOS3a3); print()
prove3(KB3a3,SOS3a3); print()

print('test d','\n------')
KB3a4 = [ {1}, {-1,2,3}, {-3,4}, {-3,5} ]
SOS3a4 = [ {-5} ]

print('KB3a4: ',end=''); pprint(KB3a4);
print('SOS3a4: ',end=''); pprint(SOS3a4); print()
prove3(KB3a4,SOS3a4); print()


test a 
------
KB3a1: [ { B } ]
SOS3a1: [ { -B } ]

Step 1: {-2}
Unsatisfiable. Number of steps: 1

test b 
------
KB3a2: [ { A }, { B, -A }, { C, -B }, { D, -C } ]
SOS3a2: [ { -D } ]

Step 1: {-4}
Step 2: {-3}
Step 3: {-2}
Step 4: {-1}
Unsatisfiable. Number of steps: 4

test c 
------
KB3a3: [ { A }, { B, C, -A }, { D, -C }, { E, -C }, { -E } ]
SOS3a3: [ { -B } ]

Step 1: {-2}
Step 2: {3, -1}
Step 3: {3}
Step 4: {4, -1}
Step 5: {5, -1}
Step 6: {4}
Step 7: {5}
Unsatisfiable. Number of steps: 7

test d 
------
KB3a4: [ { A }, { B, C, -A }, { D, -C }, { E, -C } ]
SOS3a4: [ { -E } ]

Step 1: {-5}
Step 2: {-3}
Step 3: {2, -1}
Step 4: {2}
Satisfiable. Number of steps: 4



### Part B

Now take the same clause set as in Problem Two Part B, but move the last clause into the SOS and keep the other three in KB. Run this example with prove3. 


In [12]:
KB3b1 = [{-1, -2}, {-1, 2}, {1, -2}]
SOS3b1 = [{1, 2}]

print('KB3b1: ',end=''); pprint(KB3b1);
print('SOS3b1: ',end=''); pprint(SOS3b1); print()
prove3(KB3b1,SOS3b1); print()

KB3b1: [ { -A, -B }, { B, -A }, { A, -B } ]
SOS3b1: [ { A, B } ]

Step 1: {1, 2}
Step 2: {2, -2}
Step 3: {1, -1}
Step 4: {2}
Step 5: {1}
Unsatisfiable. Number of steps: 5



### Part C

Now take the same clause set as in Problem Two Part C, but put the last clause in the SOS and the other 7 in KB.
Experiment in the same way you did in Problem Two Part C. 

In [13]:
KB3c1 = [{-1, -2, -3}, {-1, -2, 3}, {-1, 2, -3}, {-1, 2, 3}, {1, -2, -3}, {1, -2, 3}, {1, 2, -3}]
SOS3c1 = [{1, 2, 3}]

print('KB3c1: ',end=''); pprint(KB3c1);
print('SOS3c1: ',end=''); pprint(SOS3c1); print()

# prove3(KB3c1, SOS3c1, 3, True); print()
# prove3(KB3c1, SOS3c1, 4, True);prove3(CL2d, 4, True); print()
# prove3(KB3c1, SOS3c1, 5, True); print()

prove3(KB3c1, SOS3c1, 15, False); print()

# Anything past 15 seems to essentially run forever in jupyter notebooks. Although it isn't crashing my computer anymore.
# 15 runs close to a minute

KB3c1: [ { -C, -A, -B }, { C, -A, -B }, { B, -C, -A }, { B, C, -A }, { A, -C, -B }, { A, C, -B }, { A, B, -C } ]
SOS3c1: [ { A, B, C } ]

Not proven after 15 steps.



### Part D

Provide a brief statement of what you observed using the set of support strategy. Explain why you think this behavior is happening.  

From what I can tell the new prover seems to be working more efficiently than prove1(), this is because the new prover more efficiently splits up the clause and then finds/filters through the resolvents quicker.

## Problem 4 (Removing Useless Clauses) (10 pts)

Keeping the set of support strategy, in this problem you must rewrite your prover from Problem 3 as `prove4`, which
checks clauses as they are generated by resolution, and does NOT put them in the queue
if

- They are tautologies (some symbol and its negation both appear in the clause);
- They have already been generated before.

For the second, you must create a set `Visited` which is a set (hash table) containing
copies of all clauses generated at any time in the past; in order to create a set of
sets, you will need to make copies of the clauses as `frozenset`s. Here is a brief
example of how to do that:

In [14]:
S = set()
A = {1,2}

S.add( frozenset(A) )
print(S)
print( A in S)

{frozenset({1, 2})}
True


### Part A

Write `prove4` as specified; you probably want to create a separate function `isTautology(C)`. 
Test it on examples as in Problem 3, Part A, just to verify you have done it correctly. 

In [15]:
def prove4(KB, SOS, limit=30, trace=True):
    queue = SOS
    visited = set()
    steps = 0
    while queue and steps < limit:
        steps += 1
        current = queue.pop(0)
        if trace:
            print(f"Step {steps}: {current}")
        
        if not current:
            print(f"Unsatisfiable. Number of steps: {steps}")
            return
    
        if isTautology(current):
            continue
        
        for resolvent in resolveAll(KB + queue, current):
            if len(resolvent) == 0:
                print(f"Unsatisfiable. Number of steps: {steps}")
                return
            elif frozenset(resolvent) not in visited:
                if not isTautology(resolvent):
                    queue.append(resolvent)
                visited.update(frozenset(resolvent))

    if steps == limit:
        print(f"Not proven after {limit} steps.")
    else:
        print(f"Satisfiable. Number of steps: {steps}")
        
def isTautology(C):
    for literal1 in C:
        if -literal1 in C:
            return True
    return False

In [16]:
print('\ntest a','\n------')
KB3a1 = [ {2} ]
SOS3a1 = [ {-2} ]

print('KB3a1: ',end=''); pprint(KB3a1); 
print('SOS3a1: ',end=''); pprint(SOS3a1); print()
prove4(KB3a1,SOS3a1); print()

print('test b','\n------')
KB3a2 = [ {1}, {-1,2}, {-2,3}, {-3,4} ]
SOS3a2 = [ {-4} ]

print('KB3a2: ',end=''); pprint(KB3a2);
print('SOS3a2: ',end=''); pprint(SOS3a2); print()
prove4(KB3a2,SOS3a2); print()

print('test c','\n------')
KB3a3 = [ {1}, {-1,2,3}, {-3,4}, {-3,5}, {-5} ]
SOS3a3 = [ {-2} ]

print('KB3a3: ',end=''); pprint(KB3a3);
print('SOS3a3: ',end=''); pprint(SOS3a3); print()
prove4(KB3a3,SOS3a3); print()

print('test d','\n------')
KB3a4 = [ {1}, {-1,2,3}, {-3,4}, {-3,5} ]
SOS3a4 = [ {-5} ]

print('KB3a4: ',end=''); pprint(KB3a4);
print('SOS3a4: ',end=''); pprint(SOS3a4); print()
prove4(KB3a4,SOS3a4); print()


test a 
------
KB3a1: [ { B } ]
SOS3a1: [ { -B } ]

Step 1: {-2}
Unsatisfiable. Number of steps: 1

test b 
------
KB3a2: [ { A }, { B, -A }, { C, -B }, { D, -C } ]
SOS3a2: [ { -D } ]

Step 1: {-4}
Step 2: {-3}
Step 3: {-2}
Step 4: {-1}
Unsatisfiable. Number of steps: 4

test c 
------
KB3a3: [ { A }, { B, C, -A }, { D, -C }, { E, -C }, { -E } ]
SOS3a3: [ { -B } ]

Step 1: {-2}
Step 2: {3, -1}
Step 3: {3}
Step 4: {4, -1}
Step 5: {5, -1}
Step 6: {4}
Step 7: {5}
Unsatisfiable. Number of steps: 7

test d 
------
KB3a4: [ { A }, { B, C, -A }, { D, -C }, { E, -C } ]
SOS3a4: [ { -E } ]

Step 1: {-5}
Step 2: {-3}
Step 3: {2, -1}
Step 4: {2}
Satisfiable. Number of steps: 4



### Part B

Now test it on the example from Problem Three, Part B. 

In [17]:
KB4b1 = [{-1, -2}, {-1, 2}, {1, -2}]
SOS4b1 = [{1, 2}]

print('KB4b1: ',end=''); pprint(KB4b1);
print('SOS4b1: ',end=''); pprint(SOS4b1); print()
prove4(KB4b1,SOS4b1); print()

KB4b1: [ { -A, -B }, { B, -A }, { A, -B } ]
SOS4b1: [ { A, B } ]

Step 1: {1, 2}
Step 2: {2}
Step 3: {1}
Unsatisfiable. Number of steps: 3



### Part C

Now test it on the example from Problem Three, Part C, and experiment with it as
before to see how far you can search in about one minute. 

In [18]:
KB4c1 = [{-1, -2, -3}, {-1, -2, 3}, {-1, 2, -3}, {-1, 2, 3}, {1, -2, -3}, {1, -2, 3}, {1, 2, -3}]
SOS4c1 = [{1, 2, 3}]

print('KB4c1: ',end=''); pprint(KB4c1);
print('SOS4c1: ',end=''); pprint(SOS4c1); print()

prove4(KB4c1, SOS4c1, 19, False); print()

# Right around 20 is the limit where jupyter notebooks will run forever, 19 runs in under a minute

KB4c1: [ { -C, -A, -B }, { C, -A, -B }, { B, -C, -A }, { B, C, -A }, { A, -C, -B }, { A, C, -B }, { A, B, -C } ]
SOS4c1: [ { A, B, C } ]

Not proven after 19 steps.



### Part D

Summarize what you observed. Does this strategy make a significant difference?

(If you have an interest, you might try this on a similar example with 4 symbols....)

- This strategy is so far definitly the fastest, I believe it may be bottlenecked by jupyter notebooks, yet is still clearly faster than the other provers. This prover is quicker because 1. it removes taotology's(which appear often and is a major efficiency improver) and 2. remembers previous resolvents in the queue(which has been implemented in previous versions as well)

## Problem Five (Shortest-Clauses-First) (10 pts)

Now you should take your latest `prove4` and add one more refinement:  make your queue
a priority queue ordered by size, with smaller clauses to the front.

The easiest way to do this (certainly not the most efficient) is to simply sort the queue every time
you add something to the end. You can sort a list of sets/lists using as key the size of the member
set/list, as illustrated here:

In [19]:
A = [ {1,2}, {2,3,4}, {1}, {3}, {3,4}]

A.sort(reverse=True, key=(lambda x: len(x)))            # sorts in place

A

[{2, 3, 4}, {1, 2}, {3, 4}, {1}, {3}]

### Part A

Rewrite `prove4` as `prove5` with a priority queue instead of a queue, as suggested, and test it
on the same simple examples as in Part A of the previous two problems, to verify that it works as expected. 

In [20]:
def prove5(KB, SOS, limit=30, trace=True):
    queue = SOS
    visited = set()
    steps = 0
    while queue and steps < limit:
        steps += 1
        current = queue.pop(0)
        if trace:
            print(f"Step {steps}: {current}")
        
        if not current:
            print(f"Unsatisfiable. Number of steps: {steps}")
            return
    
        if isTautology(current):
            continue
        
        for resolvent in resolveAll(KB + queue, current):
            if len(resolvent) == 0:
                print(f"Unsatisfiable. Number of steps: {steps}")
                return
            elif frozenset(resolvent) not in visited:
                if not isTautology(resolvent):
                    queue.append(resolvent)
                    queue.sort(reverse=True, key=(lambda x: len(x)))
                visited.update(frozenset(resolvent))

    if steps == limit:
        print(f"Not proven after {limit} steps.")
    else:
        print(f"Satisfiable. Number of steps: {steps}")
        
def isTautology(C):
    for literal1 in C:
        if -literal1 in C:
            return True
    return False

In [21]:
print('\ntest a','\n------')
KB3a1 = [ {2} ]
SOS3a1 = [ {-2} ]

print('KB3a1: ',end=''); pprint(KB3a1); 
print('SOS3a1: ',end=''); pprint(SOS3a1); print()
prove5(KB3a1,SOS3a1); print()

print('test b','\n------')
KB3a2 = [ {1}, {-1,2}, {-2,3}, {-3,4} ]
SOS3a2 = [ {-4} ]

print('KB3a2: ',end=''); pprint(KB3a2);
print('SOS3a2: ',end=''); pprint(SOS3a2); print()
prove5(KB3a2,SOS3a2); print()

print('test c','\n------')
KB3a3 = [ {1}, {-1,2,3}, {-3,4}, {-3,5}, {-5} ]
SOS3a3 = [ {-2} ]

print('KB3a3: ',end=''); pprint(KB3a3);
print('SOS3a3: ',end=''); pprint(SOS3a3); print()
prove5(KB3a3,SOS3a3); print()

print('test d','\n------')
KB3a4 = [ {1}, {-1,2,3}, {-3,4}, {-3,5} ]
SOS3a4 = [ {-5} ]

print('KB3a4: ',end=''); pprint(KB3a4);
print('SOS3a4: ',end=''); pprint(SOS3a4); print()
prove5(KB3a4,SOS3a4); print()


test a 
------
KB3a1: [ { B } ]
SOS3a1: [ { -B } ]

Step 1: {-2}
Unsatisfiable. Number of steps: 1

test b 
------
KB3a2: [ { A }, { B, -A }, { C, -B }, { D, -C } ]
SOS3a2: [ { -D } ]

Step 1: {-4}
Step 2: {-3}
Step 3: {-2}
Step 4: {-1}
Unsatisfiable. Number of steps: 4

test c 
------
KB3a3: [ { A }, { B, C, -A }, { D, -C }, { E, -C }, { -E } ]
SOS3a3: [ { -B } ]

Step 1: {-2}
Step 2: {3, -1}
Step 3: {4, -1}
Step 4: {5, -1}
Step 5: {3}
Step 6: {4}
Step 7: {5}
Unsatisfiable. Number of steps: 7

test d 
------
KB3a4: [ { A }, { B, C, -A }, { D, -C }, { E, -C } ]
SOS3a4: [ { -E } ]

Step 1: {-5}
Step 2: {-3}
Step 3: {2, -1}
Step 4: {2}
Satisfiable. Number of steps: 4



### Part B
Again, test your new prover on the example from Part B of the previous two problems. 

In [22]:
KB4b1 = [{-1, -2}, {-1, 2}, {1, -2}]
SOS4b1 = [{1, 2}]

print('KB4b1: ',end=''); pprint(KB4b1);
print('SOS4b1: ',end=''); pprint(SOS4b1); print()
prove5(KB4b1,SOS4b1); print()

KB4b1: [ { -A, -B }, { B, -A }, { A, -B } ]
SOS4b1: [ { A, B } ]

Step 1: {1, 2}
Step 2: {2}
Step 3: {1}
Unsatisfiable. Number of steps: 3



### Part C
Again, test your new prover on the example from Part C of the previous two problems. 

In [23]:
KB4c1 = [{-1, -2, -3}, {-1, -2, 3}, {-1, 2, -3}, {-1, 2, 3}, {1, -2, -3}, {1, -2, 3}, {1, 2, -3}]
SOS4c1 = [{1, 2, 3}]

print('KB4c1: ',end=''); pprint(KB4c1);
print('SOS4c1: ',end=''); pprint(SOS4c1); print()

prove5(KB4c1, SOS4c1, 30, False); print()

# Right around 35 is the limit where jupyter notebooks will run forever, 30 will run in a little bit under a minute/

KB4c1: [ { -C, -A, -B }, { C, -A, -B }, { B, -C, -A }, { B, C, -A }, { A, -C, -B }, { A, C, -B }, { A, B, -C } ]
SOS4c1: [ { A, B, C } ]

Not proven after 30 steps.



### Part D

What did you observe?  Did this make a significant difference in the amount of time it took to complete
the proof?  

Finally, do you think these refinements would reduce the exponential search space for such
problems to polynomial size, or
is exponential growth still a problem? (Hint: think about what you learned in your algorithms or
complexity class about the Satisfiability Problem.)

- Sorting(I believe) has made the biggest difference yet between provers. With this prover being able to run up to limit=30 without too much issue. Elimanting the smallest clauses first is clearly efficient for this problem.
- While I'm not sure, I think the search space would decrease to polynomial size. Between remembering visited resolvents, filtering out tautologies, and sorting, the prover does need to search through nearly as many options as before, resulting in a smaller search space.

## Problem Six (10 pts)

In this problem, we will use the fastest method, `prove5`, to solve some word problems which can
be encoded as problems in propositional logic. For each, give the encoding of the statements
in the problem as symbols. (You might want to review pp.32-33 in the textbook before starting.)

### Part A

$$ (Accomplice \implies Car) \land ((\neg Accomplice \land \neg Keys) \lor (Accomplice \land Keys)) \land Keys $$  
Accomplice = A, Car = B, Keys = C  
Conversion to CNF:

$$\begin{aligned}
        (\neg A \lor C) \land ( ((\neg A \land \neg K) \lor A) \land ((\neg A \land \neg K) \lor K)) \land K  \\
        (\neg A \lor C) \land ( ((\neg A \lor A) \land (A \lor \neg K)) \land ((\neg A \lor K) \land (K \lor \neg K))) \land K  \\
        (\neg A \lor C) \land (A \lor \neg K) \land (\neg A \lor K) \land K \\
  \end{aligned}$$

Trying to prove C...
$$\begin{aligned}
        (\neg A \lor C) \land (A \lor \neg K) \land (\neg A \lor K) \land K \land \neg C \\
  \end{aligned}$$
  
Resolution Proof:
$$\begin{aligned}
        Res(2, 4) = A_6 \\
        Res(1, 6) = C_7 \\
        Res(5, 7) = ()_8 \\
  \end{aligned}$$

In [24]:
# A = 1, C = 2, K = 3
KB = [{-1, 2}, {1, -3}, {-1, 3}, {3}]
SOS = [{-1}]
prove5(KB, SOS)

Step 1: {-1}
Step 2: {-3}
Unsatisfiable. Number of steps: 2


### Part B

A man and a woman are talking. “I am a
man” said the person with black hair. “I am a woman” said the person with white hair.
At least one of them is lying.  Formalize the puzzle using propositional logic and show using resolution that both of them are lying.

<!-- Attempt one lol -->
<!-- Man = M, Woman = F, Black hair = B, White hair = W
(White hair implies female) and (Black hair implies male) and (at least one is lying)
$$\begin{aligned}
        (W \implies F) \land (B \implies M) \land ((\neg W \implies \neg F) \lor (\neg B \implies \neg M)) \\
        (\neg W \lor F) \land (\neg B \lor M) \land ((W \lor \neg F) \lor (B \lor \neg M)) \\
        (\neg W \lor F) \land (\neg B \lor M) \land (W \lor \neg F \lor B \lor \neg M)
  \end{aligned}$$
 
Trying to Prove both are lying:
$$\begin{aligned}
    \neg ((\neg W \implies \neg F) \land (\neg B \implies \neg M)) \\
    \neg((W \lor \neg F) \land (B \lor \neg M)) \\
    (\neg (W \lor \neg F) \lor \neg(B \lor \neg M)) \\
    (\neg W \land F) \lor (\neg B \land M) \\
    ((\neg W \land F) \lor \neg B) \land ((\neg W \land F) \lor M) \\
    (\neg B \lor \neg W) \land (\neg B \lor F) \land (M \lor \neg W) \land (M \lor F)
\end{aligned}$$

CNF: $$ (\neg W \lor F) \land (\neg B \lor M) \land (W \lor \neg F \lor B \lor \neg M) \land (W \lor \neg F) \land (B \lor \neg M)  $$ -->

WT = white hair tells truth  
WM = white hair is male  
BT = black hair tells truth  
BM = black hair is male  

= (black hair telling truth implies their male) & (white hair telling truth implies their not male) & (at least one is lying) & (if white hair is male, black hair is not male, or vice versa)

$$\begin{aligned}
    \equiv (BT \implies BM) \land (WT \implies \neg WM) \land (\neg WT \lor \neg BT) \land ((WM \land \neg BM) \lor (\neg WM \land BM)) \\
    \equiv (\neg BT \lor BM) \land (\neg WT \lor \neg WM) \land (\neg WT \lor \neg BT) \land (\neg WM \lor \neg BM) \land (WM \lor BM)
\end{aligned}$$

Trying to prove both are lying...  
$$\begin{aligned}
    \equiv (\neg BT \lor BM) \land (\neg WT \lor \neg WM) \land (\neg WT \lor \neg BT) \land (\neg WM \lor \neg BM) \land (WM \lor BM) \land \neg(\neg WT \land \neg BT) \\
    \equiv (\neg BT \lor BM) \land (\neg WT \lor \neg WM) \land (\neg WT \lor \neg BT) \land (\neg WM \lor \neg BM) \land (WM \lor BM) \land WT \land BT \\
\end{aligned}$$

Resolution Proof:
$$\begin{aligned}
        Res(3, 6) = (\neg BT)_8 \\
        Res(7, 8) = ()_9
  \end{aligned}$$

In [25]:
KB = [{-1, 2}, {-3, -4}, {-1, -3}, {2, 4}, {-2, -4}]
SOS = [{3}, {1}]
prove5(KB, SOS)

Step 1: {3}
Step 2: {1}
Unsatisfiable. Number of steps: 2


### Part C

Three boxes are presented to you. One contains gold, the other two are empty.
Each box has imprinted on it a clue as to its contents; the clues are:

- Box 1 “The gold is not here”
- Box 2 “The gold is not here”
- Box 3 “The gold is in Box 2”

Only one message is true; the other two are false. Which box has the gold?

Formalize the puzzle using propositional logic and find the solution using resolution.   
    
B1 = Gold in box one, B2 = Gold in box two, B3 = Gold in box three  
One box contains gold...
$$\begin{aligned}
   \equiv (B1 \land \neg B2 \land \neg B3) \lor (\neg B1 \land B2 \land \neg B3) \lor (\neg B1 \land \neg B2 \land B3) \\
   \equiv (\neg B1 \lor \neg B2) \land (\neg B1 \lor \neg B3) \land (\neg B2 \lor \neg B3) \land (B1 \lor B2 \lor B3)
\end{aligned}$$

One message true, the others false...
$$\begin{aligned}
   \equiv (\neg B1 \land B2 \land \neg B2) \lor (B1 \land \neg B2 \land \neg B2) \lor (B1 \land B2 \land B2) \\
   \equiv F \lor (B1 \land \neg B2) \lor (B1 \land B2) \\
   \equiv (B1 \land \neg B2) \lor (B1 \land B2) \\
   \equiv B1
\end{aligned}$$

Proof:
$$\begin{aligned}
   \equiv (\neg B1 \lor \neg B2) \land (\neg B1 \lor \neg B3) \land (\neg B2 \lor \neg B3) \land (B1 \lor B2 \lor B3) \land \neg B1
\end{aligned}$$

Resolution Proof:
$$\begin{aligned}
        Res(4, 5) = (B2 \lor B3)_6 \\
        Res(3, 7) = ()_7
  \end{aligned}$$

Box 3 has the gold?

In [26]:
# M = 1, F = 2, B = 3, W = 4
KB = [{1, -2, -3}, {-1, 2, -3}, {1, 2, -3}]
# SOS = [{-3, -4}, {-3, 2}, {1, -4}, {1, 2}]
SOS = [{-1}]
prove5(KB, SOS)

Step 1: {-1}
Step 2: {-3, -2}
Step 3: {2, -3}
Step 4: {-3, -1}
Step 5: {1, -3}
Step 6: {1, -3}
Step 7: {-3, -2}
Step 8: {2, -3}
Step 9: {2, -3}
Step 10: {2, -3}
Step 11: {-3, -1}
Step 12: {1, -3}
Step 13: {1, -3}
Step 14: {1, -3}
Step 15: {1, -3}
Step 16: {-3, -2}
Step 17: {2, -3}
Step 18: {2, -3}
Step 19: {2, -3}
Step 20: {2, -3}
Step 21: {2, -3}
Step 22: {-3, -1}
Step 23: {1, -3}
Step 24: {1, -3}
Step 25: {1, -3}
Step 26: {1, -3}
Step 27: {1, -3}
Step 28: {1, -3}
Step 29: {-3, -2}
Step 30: {2, -3}
Not proven after 30 steps.


## Problem Seven (First-Order Logic) (5 pts)

Do problem 3.1 from your textbook. You may provide your solution using ASCII text; by hand-writing, scanning, and pasting; or (preferably) in Latex.  
**a.** $$ \forall x male(x) \iff \neg female(x) $$
**b.** $$ \forall x \forall y \exists z father(x) \iff male(x) \land child(x, y, z) $$
**c.** $$ \forall x \forall y siblings(x,y) \iff (\exists z father(z,x) \land father(z,y)) \lor (\exists z mother(z, x) \land mother(z, y)) $$
**d.** $$ \forall x \forall y \forall z parents(x, y, z) \iff father(x, z) \land mother(y, z) $$
**e.** $$ \forall x \forall y uncle(x, y) \iff \exists z \exists u child(y, z, u) \land siblings(z, x) \land male(x) $$
**f.** $$ \forall x \forall y ancestor(x, y) \iff \exists z child(y, x, z) \lor \exists u \exists v child(u,x,v) \land ancestor(u,y) $$

## Problem Eight (First-Order Logic)  (5 pts)

Do problem 3.2 from your textbook. You may provide your solution using ASCII text; by hand-writing, scanning, and pasting; or (preferably) in Latex.  
**a.** $$ \forall x \exists y \exists z father(y, x) \land mother(z, x) $$
**b.** $$ \exists x \exists y child(y, x, z) $$
**c.** $$ \forall x bird(x) \iff flies(x) $$
**d.** $$ \exists x \exists y \exists z animal(x) \land animal(y) \land eats(x, y) \land eats(y, z) \land grain(z) $$
**e.** $$ \forall x animals(x) \iff \exists y (eats(x,y) \land (plant(y) \lor (animal(y) \land \exists z plant(z) \land eats(y,z) \land smaller(y, x)))) $$

## Problem Nine (First-Order Logic)  (5 pts)

Do problem 3.4 from your textbook. You may provide your solution using ASCII text; by hand-writing, scanning, and pasting; or (preferably) in Latex.  
**1.**$$ \forall x \forall y $$ $$ x < y \lor x = y \lor x > y $$
**2.**$$ \forall x \forall y $$ $$ x < y \implies \neg y < x $$
**3.**$$ \forall x \forall y \forall z $$ $$ (y < z \implies x < z) \land x < y $$

## Problem Ten (First-Order Logic)  (5 pts)

Do problem 3.5 from your textbook. You may provide your solution using ASCII text; by hand-writing, scanning, and pasting; or (preferably) in Latex.  
**a.**
MGU: $$x/(fz), u/f(y)$$  
Term: $$p(f(z), f(y))$$

**b.** not unifiable

**c.**  
MGU: $$ x/cos(y), z/4 - (7 * cos(y)) $$  
Term: $$ cos(y) = 4 - (7*cos(y)) $$

**d.** not unifiable

**e.**  
MGU: 
$$ x/g(w,w), \\ y/g(g(w,w), g(w,w)), \\ z/g(g(g(w,w), g(w,w)), g(g(w,w), g(w,w))) $$
$$ u/f(g(w,w),g(g(w,w), g(w,w)), g(g(g(w,w), g(w,w)), g(g(w,w),g(w,w)))) $$
Term: 
$$ p(f(g(w,w),g(g(w,w), g(w,w)), g(g(g(w,w), g(w,w)), g(g(w,w),g(w,w)))), \\ f(g(w,w),g(g(w,w), g(w,w)), g(g(g(w,w), g(w,w)), g(g(w,w),g(w,w)))))$$
